# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [239]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [240]:
!pip install gmaps


In [241]:
!jupyter nbextension enable --py --sys-prefix gmaps


Enabling notebook extension jupyter-gmaps/extension...
      - Validating: OK


In [242]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Google developer API key
from api_keys import g_key

# Access maps with unique API key
gmaps.configure(api_key="ENTER KEY HERE")

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [243]:
data_file = "../output_data/cities.csv"
cities_df = pd.read_csv(data_file)


cities_df.dropna()
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [244]:
gmaps.configure(api_key="ENTER KEY HERE")


In [245]:
locations = cities_df[["Lat", "Lng"]]

humidity = cities_df["Humidity"].astype(float)

In [246]:
# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=130,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [247]:
newdf = pd.DataFrame(cities_df)

max_temp = (newdf["Max Temp"] < 80) & (newdf["Max Temp"] > 70)
windspeed = newdf["Wind Speed"] < 10
nocloudy = newdf["Cloudiness"] == 0

newnewdf = newdf[max_temp & windspeed & nocloudy ]
newnewdf

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,7,opuwo,0,NaN,1558378523,23,-18.06,13.84,72.06,5.44
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [248]:
hotel_df = pd.DataFrame(newnewdf, columns=["City","Country","Lat","Lng"])

hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
7,opuwo,NaN,-18.06,13.84,
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [249]:
pprint(response.json(), depth=1)


Pretty printing has been turned OFF


In [250]:
hotel_df.head()


,City,Country,Lat,Lng,Hotel Name
7,opuwo,NaN,-18.06,13.84,
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,


In [251]:
params={
    "radius": 5000,
    "keyword": "hotel",
    "key": g_key
}

for index,row in hotel_df.iterrows():
    lat=row["Lat"]
    lng=row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [252]:
hotel_name

[nan, 'Shandrani Beachcomber Resort & Spa', 'Qaanaaq Hotel', nan, 'Signature Lux Hotel by ONOMO, Waterfront', 'Villa Enata', 'Barrack Point Holidays', 'Opuwo Country Lodge', nan, 'The guest house "The Tale here"', 'Hotel Astoria', 'Enjoy Chiloé. Hotel De La Isla', 'Muri Beach Club Hotel Rarotonga', nan, 'Hotel Gobernador', 'Renaissance Albany Hotel', 'Gostinitsa "Mariya"', 'Los Cauquenes Resort & Spa', 'Laguna Beach Hotel & Spa', 'Отель Купеческий', 'Milky Resort', 'Esplanade Hotel', 'Domaine de Rochevilaine', 'Salinas do Maragogi All Inclusive Resort', nan, 'โรงแรมกันตังปาร์ค(Kantang Park Hotel)', "AMAAN'S HOUSE", 'Hotel Cabo de Hornos', "Pension Maro'i", 'The Woodbridge', 'Goba Lodge', 'The Westin Qingdao', 'Quarter Deck Inn', nan, nan, 'Long House Hotel Bethel', 'AmericInn by Wyndham, Fulton Clinton', 'Huset ved havet', nan, nan, 'Nautilus Cottage', "Best Western Plus St. John's Airport Hotel and Suites", 'The Explorer Hotel', nan, 'Hotel Solymar Galapagos', 'Hotel Rosita', nan, nan

In [253]:
hotel_df.dropna()

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,Escarpas Resort
88,cidreira,BR,-30.17,-50.22,Pousada Rosa
138,morondava,MG,-20.30,44.28,Kimony Resort Hotel
139,vaini,IN,15.34,74.49,Dandeli Dreams Jungle resorts
176,nador,MA,35.17,-2.93,Hotel Beautiful
297,mogok,MM,22.92,96.51,Mount Mogok Hotel
444,birjand,IR,32.86,59.22,Sepehr Hotel


In [254]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
7,opuwo,NaN,-18.06,13.84,Opuwo Country Lodge
70,ponta do sol,BR,-20.63,-46.00,Escarpas Resort
88,cidreira,BR,-30.17,-50.22,Pousada Rosa
138,morondava,MG,-20.30,44.28,Kimony Resort Hotel
139,vaini,IN,15.34,74.49,Dandeli Dreams Jungle resorts


In [255]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [256]:
# Add marker layer ontop of heat map

markers =gmaps.marker_layer(locations,info_box_content =hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))